In [2]:
import pandas as pd
import numpy as np

In [3]:
cpi_df = pd.read_csv('/home/datahouse1/liujin/CPI_offtarget/datasets/CPI_data_cls/raw_data/all_cpi_data_pro_final2.csv')
print(cpi_df.shape)
cpi_df.head(2)

(2102767, 4)


,uniprot_id,smiles,label,canonical_smiles
0,Q7X1P7,CO[C@@H]1[C@@H](OC(N)=O)[C@@H](O)[C@H](Oc2ccc3...,0,CO[C@@H]1[C@@H](OC(N)=O)[C@@H](O)[C@H](Oc2ccc3...
1,Q7X1P7,CCNC(=O)Nc1cn2c(-c3ncc(C(F)(F)F)cn3)cc(-c3cccn...,1,CCNC(=O)Nc1cn2c(-c3ncc(C(F)(F)F)cn3)cc(-c3cccn...


In [4]:
print(cpi_df['uniprot_id'].nunique())
print(cpi_df['canonical_smiles'].nunique())
print(cpi_df['smiles'].nunique()) #不同的smiles，可能对应相同的canonical_smiles

cpi_df2 = cpi_df.drop_duplicates(subset=['uniprot_id', 'smiles']) #(2102767, 5)
print(cpi_df2.shape)

7258
1255941
1312725
(2102767, 4)


In [5]:
#读取蛋白质序列文件，txt格式
import json
protein_seq = json.load(open('/home/datahouse1/liujin/CPI_offtarget/datasets/CPI_data_cls/raw_data/protein_id_seq_new.txt', 'r'))
print(len(protein_seq))

7258


#### 将数据修改为模型所需要的格式

In [6]:
compound_list = cpi_df['canonical_smiles'].unique()
print(len(compound_list)) #化合物数目为1181909
#给每一条smiles进行编号
compound_dict_id = {}
# for i in range(len(compound_list)):
#     id = 'compound_'+str(i)
#     compound_dict_id[compound_list[i]] = id
# print(len(compound_dict_id))
for i in range(len(compound_list)):
    id = 'compound_'+str(i)
    compound_dict_id[id] = compound_list[i]
print(len(compound_dict_id))

1255941
1255941


In [7]:
#把化合物id字典写入txt文件中
#将化合物id列表写入txt文件
# 这里存的字典，键是id，值是smiles
with open('/home/datahouse1/liujin/CPI_offtarget/datasets/CPI_data_cls/train/compounds.txt','w') as f:
    json.dump(compound_dict_id,f)

In [10]:
compound_id_smile_map = {}
for i in range(len(compound_list)):
    id = 'compound_'+str(i)
    compound_id_smile_map[compound_list[i]] = id
print(len(compound_id_smile_map))

cpi_df['comp_id'] = cpi_df['canonical_smiles'].map(compound_id_smile_map) #这里用来映射的compound_dict_id键是smiles，值是id
cpi_df.head(5)

1255941


,uniprot_id,smiles,label,canonical_smiles,comp_id
0,Q7X1P7,CO[C@@H]1[C@@H](OC(N)=O)[C@@H](O)[C@H](Oc2ccc3...,0,CO[C@@H]1[C@@H](OC(N)=O)[C@@H](O)[C@H](Oc2ccc3...,compound_0
1,Q7X1P7,CCNC(=O)Nc1cn2c(-c3ncc(C(F)(F)F)cn3)cc(-c3cccn...,1,CCNC(=O)Nc1cn2c(-c3ncc(C(F)(F)F)cn3)cc(-c3cccn...,compound_1
2,Q7X1P7,CCC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3cccnc3)cc2n1,0,CCC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3cccnc3)cc2n1,compound_2
3,Q7X1P7,CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3ccn(CCF)c(=...,0,CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3ccn(CCF)c(=...,compound_3
4,Q7X1P7,CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3ccn(CC)c(=O...,1,CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3ccn(CC)c(=O...,compound_4


In [11]:
cpi_df_choose = cpi_df[['uniprot_id', 'comp_id', 'label']]
print(cpi_df_choose.shape)
cpi_df_choose.head(5)

(2102767, 3)


,uniprot_id,comp_id,label
0,Q7X1P7,compound_0,0
1,Q7X1P7,compound_1,1
2,Q7X1P7,compound_2,0
3,Q7X1P7,compound_3,0
4,Q7X1P7,compound_4,1


In [12]:
cpi_df_choose.to_csv('/home/datahouse1/liujin/CPI_offtarget/datasets/CPI_data_cls/train/affinity.tsv',sep='\t', index=False)

In [13]:
affinity = open('/home/datahouse1/liujin/CPI_offtarget/datasets/CPI_data_cls/train/affinity.tsv','r').readlines()
print(len(affinity))

2102768
